In [2]:
import pandas as pd
import numpy as np
from anndata import read_h5ad, AnnData
import scanpy as sc
import json

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_decomposition import PLSRegression

import altair as alt
from altair_saver import save as alt_save

In [3]:
metmap_tissues = snakemake.params['metmap_tissues']
tm_to_metmap = snakemake.params['tm_to_metmap']

In [4]:
curr_fold = int(snakemake.wildcards["fold"])
curr_fold

In [5]:
kfold_df = pd.read_csv(snakemake.input['kfold_indices'])
kfold_train_df = kfold_df.loc[(kfold_df["fold"] == curr_fold) & (kfold_df["set"] == "train")]
kfold_test_df = kfold_df.loc[(kfold_df["fold"] == curr_fold) & (kfold_df["set"] == "test")]

In [6]:
kfold_train_df.head()

In [7]:
ccle_adata = read_h5ad(snakemake.input['ccle_exp'])

In [8]:
# Preprocess the gene expression data based on the current wildcards
gexp_transform = snakemake.wildcards["gexp_transform"]
if gexp_transform == "tpm":
    sc.pp.normalize_total(ccle_adata, target_sum=1e4)
elif gexp_transform == "log1p_tpm":
    sc.pp.normalize_total(ccle_adata, target_sum=1e4)
    sc.pp.log1p(ccle_adata)
elif gexp_transform == "log1p_tpm_scale":
    sc.pp.normalize_total(ccle_adata, target_sum=1e4)
    sc.pp.log1p(ccle_adata)
    sc.pp.scale(ccle_adata, max_value=10)

In [9]:
mm_all_df = pd.read_excel(snakemake.input['mm_potential'], sheet_name=f"metp500.all5", index_col=0)

In [10]:
train_celllines = kfold_train_df["cellline"].values.tolist()
test_celllines = kfold_test_df["cellline"].values.tolist()

In [11]:
# Need to take union of significantly differentially expressed genes in the training set.

deseq_files = dict(zip(metmap_tissues, snakemake.input[:len(metmap_tissues)]))
deseq_dfs = {}

significance_level = 0.05
fc_threshold = float(snakemake.wildcards["fc_threshold"])

deseq_significant_union = set()

for tissue, deseq_file in deseq_files.items():
    tissue_deseq_df = pd.read_csv(deseq_file, index_col=0)
    tissue_deseq_df["significant"] = tissue_deseq_df.apply(lambda row: row['padj'] <= significance_level and abs(row['log2FoldChange']) >= fc_threshold, axis='columns')
    # Filter to keep only the significantly differentially expressed genes
    tissue_deseq_df = tissue_deseq_df.loc[tissue_deseq_df["significant"]]
    
    deseq_dfs[tissue] = tissue_deseq_df
    
    deseq_significant_union = deseq_significant_union.union(set(tissue_deseq_df.index.values.tolist()))

deseq_signficant_genes = list(deseq_significant_union)
len(deseq_signficant_genes)

In [12]:
tissue_train_test_X = {
    "train": ccle_adata[train_celllines, deseq_signficant_genes].X,
    "test": ccle_adata[test_celllines, deseq_signficant_genes].X,
}

Metastatic potential: "Data are presented on a log10 scale, range from -4 ~ 4."
- <= -4: non-metastatic
- -4~-2: (weakly) metastatic, but with low confidence
- \>= -2: metastatic, with higher confidence



In [13]:
def met_potential_encode(arr):
    non_metastatic_mask = (arr <= -4)
    weakly_metastatic_mask = (arr > -4) & (arr < -2)
    metastatic_mask = (arr >= -2)
    
    arr[non_metastatic_mask] = 0
    arr[weakly_metastatic_mask] = 1
    arr[metastatic_mask] = 2
    
    return arr
    return np.power(np.repeat(10, arr.shape[0]), arr)

def met_potential_decode(arr):
    return arr
    return np.log10(np.clip(arr, a_min=0.000000001, a_max=None))

In [14]:
# Make a dictionary mapping tissue type to training and testing metastatic potential values.
# These will become the response variables for PLSRegression.
# These should be ordered according to the ordering of cell lines in kfold_train_df and kfold_test_df

train_y = []
test_y = []
for tissue in metmap_tissues:
    mm_tissue = tm_to_metmap[tissue]
    mm_tissue_df = pd.read_excel(snakemake.input['mm_potential'], sheet_name=f"metp500.{mm_tissue}", index_col=0)
    
    mm_tissue_train_df = mm_tissue_df.loc[train_celllines]
    mm_tissue_test_df = mm_tissue_df.loc[test_celllines]
    
    train_y.append(met_potential_encode(mm_tissue_train_df["mean"].values))
    test_y.append(met_potential_encode(mm_tissue_test_df["mean"].values))

tissue_train_test_y = {
    "train": np.stack(train_y, axis=-1),
    "test": np.stack(test_y, axis=-1)
}

In [15]:
n_components = int(snakemake.wildcards["num_pc"])

## PLSRegression

In [16]:
X_train = tissue_train_test_X["train"]
Y_train = tissue_train_test_y["train"]

X_test = tissue_train_test_X["test"]
Y_test = tissue_train_test_y["test"]

pls2 = PLSRegression(n_components=n_components)
pls2.fit(X_train, Y_train)

# Predict on test (held out) data
Y_pred = pls2.predict(X_test)

In [17]:
model_results = {}
for tissue_i, tissue in enumerate(metmap_tissues):
    Y_pred_tissue = met_potential_decode(Y_pred.T[tissue_i])
    Y_test_tissue = met_potential_decode(Y_test.T[tissue_i])
    #print(Y_pred_tissue)
    #print(Y_test_tissue)
    model_results[tissue] = {
        "r2": r2_score(Y_pred_tissue, Y_test_tissue),
        "mse": mean_squared_error(Y_pred_tissue, Y_test_tissue)
    }

In [18]:
model_results

In [19]:
with open(snakemake.output["model_test_results"], "w") as f:
    json.dump(model_results, f)

In [20]:
# Predict on training data
Y_train_pred = pls2.predict(X_train)

In [21]:
model_train_results = {}
for tissue_i, tissue in enumerate(metmap_tissues):
    Y_pred_tissue = met_potential_decode(Y_train_pred.T[tissue_i])
    Y_train_tissue = met_potential_decode(Y_train.T[tissue_i])
    model_train_results[tissue] = {
        "r2": r2_score(Y_pred_tissue, Y_train_tissue),
        "mse": mean_squared_error(Y_pred_tissue, Y_train_tissue)
    }

In [22]:
model_train_results

In [23]:
with open(snakemake.output["model_train_results"], "w") as f:
    json.dump(model_train_results, f)

## Basic deep learning model

In [24]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [25]:
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train).type(torch.LongTensor)

X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test).type(torch.LongTensor)

In [26]:
class GeneExpressionDataset(Dataset):
    # https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files
    def __init__(self, X, Y, tissue_i):
        self.X = X
        self.Y = Y
        self.tissue_i = tissue_i
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return (self.X[i], self.Y[i][self.tissue_i])
    

In [27]:
num_features = X_train.shape[1]
num_classes = 3

num_epochs = 150

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes),
            nn.ReLU()
        )
    
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [30]:
model = NeuralNetwork().to(device)
print(model)

In [31]:
train_acc = []
train_loss = []
test_pred = []

In [32]:
def train(dataloader, model, loss_fn, optimizer, epoch_i, tissue):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 16 == 0:
            loss, current = loss.item(), batch * len(X)
            #print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            train_loss.append((epoch_i + (current / size), loss, tissue))

In [33]:
def test(dataloader, model, epoch_i, tissue):
    global test_pred
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    np_y = []
    np_pred = []
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            if num_epochs - 1 == epoch_i:
                np_y += y.numpy().tolist()
                np_pred += pred.argmax(1).numpy().tolist()
                print(np_y, np_pred)
        
    test_pred += list(zip(np_y, np_pred, [tissue] * len(np_y)))
    test_loss /= size
    correct /= size
    #print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    train_acc.append((epoch_i, correct, tissue))

In [34]:
for tissue_i, tissue in enumerate(metmap_tissues):
    train_dataloader = DataLoader(GeneExpressionDataset(X_train, Y_train, tissue_i), batch_size=64, shuffle=True)
    test_dataloader = DataLoader(GeneExpressionDataset(X_test, Y_test, tissue_i), batch_size=64, shuffle=True)
    
    model = NeuralNetwork().to(device)
    
    unique, counts = np.unique(Y_train.T[tissue_i], return_counts=True)
    
    print((counts / counts.sum()))
    inverse_class_dist = 1 / (counts / counts.sum())
    inverse_class_dist = inverse_class_dist / inverse_class_dist.sum()
    inverse_class_dist
    
    #loss_fn = nn.CrossEntropyLoss(weight=torch.tensor(inverse_class_dist, dtype=torch.float32))
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=5e-4)
    
    for t in range(num_epochs):
        #print(f"Epoch {t+1}\n-------------")
        train(train_dataloader, model, loss_fn, optimizer, t, tissue)
        test(test_dataloader, model, t, tissue)
    print("Done!")

In [35]:
train_acc_df = pd.DataFrame(data=train_acc, columns=["epoch", "accuracy", "tissue"])
train_loss_df = pd.DataFrame(data=train_loss, columns=["epoch", "loss", "tissue"])
test_pred_df = pd.DataFrame(data=test_pred, columns=["true", "pred", "tissue"])
test_pred_df["bin"] = test_pred_df.apply(lambda row: f"{row['pred']}.{row['true']}", axis='columns')

In [36]:
test_pred_df

In [37]:
plot = alt.Chart(train_acc_df).mark_line().encode(
    x=alt.X("epoch:Q"),
    y=alt.Y("accuracy:Q", scale=alt.Scale(domain=[0, 1])),
    column=alt.Column("tissue:N")
).properties(
    title=f"Accuracy on the test set, fold {curr_fold}",
    width=200
)

plot

In [38]:
alt_save(plot, snakemake.output["nn_accuracy"])

In [39]:
plot = alt.Chart(train_loss_df).mark_line().encode(
    x=alt.X("epoch:Q"),
    y=alt.Y("loss:Q"),
    column=alt.Column("tissue:N")
).properties(
    title=f"Loss on the training set, fold {curr_fold}",
    width=200
)
plot

In [40]:
alt_save(plot, snakemake.output["nn_loss"])

In [41]:
plot = alt.Chart(test_pred_df).mark_rect().encode(
    x=alt.X("pred:O", scale=alt.Scale(domain=[0, 1, 2])),
    y=alt.Y("true:O", scale=alt.Scale(domain=[2, 1, 0])),
    color=alt.Color("count(bin):Q"),
    column=alt.Column("tissue:N")
).properties(
    title=f"Confusion matrix, fold {curr_fold}",
    width=200,
    height=200
)
plot

In [42]:
alt_save(plot, snakemake.output["nn_confusion"])